## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Importing Data

In [2]:
path = r'C:\Users\bees\Instacart Basket Analysis'

In [3]:
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_agg.pkl'))

In [4]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

## Questions

### Question 4: Wrangle the data so that it follows consistent logic
Renaming columns and dropping columns that aren't needed

In [5]:
df_customers.shape

(206209, 10)

In [6]:
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# renaming First Name to first_name
df_customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)

# renaming Surnam to last_name
df_customers.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

# renaming Gender to gender
df_customers.rename(columns = {'Gender' : 'gender'}, inplace = True)

# renaming STATE to state
df_customers.rename(columns = {'STATE' : 'state'}, inplace = True)

# renaming Age to age
df_customers.rename(columns = {'Age' : 'age'}, inplace = True)

# renaming n_dependants to number_of_dependants
df_customers.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

# renaming fam_status to marital_status
df_customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

### Question 5: Complete the fundamental data quality and consistency checks

Checking for:
missing values, duplicates, and mixed-type data

In [9]:
# checking for mixed-type data
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) >0:
        print(col)

first_name


In [10]:
df_customers.dtypes

user_id                  int64
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [11]:
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [12]:
ords_prods_merged['user_id'] = ords_prods_merged['user_id'].astype('str')

In [13]:
# checking for missing values
df_customers.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependants        0
marital_status              0
income                      0
dtype: int64

In [14]:
# looking for duplicates
customers_dups = df_customers[df_customers.duplicated()]

In [15]:
customers_dups

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income


### Question 6: Combine customer data with the rest of prepared data

In [16]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [17]:
ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_label,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_order,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [18]:
df_customers.shape

(206209, 10)

In [19]:
ords_prods_merged.shape

(32404859, 22)

In [20]:
# merging the two dfs using 'user_id' as the key
ords_prods_cust = ords_prods_merged.merge(df_customers, on = 'user_id')

In [21]:
ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [22]:
ords_prods_cust.shape

(32404859, 31)

In [23]:
ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_label,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_order,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


### Question 8: Export

In [24]:
ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_combined.pkl'))